In [2]:
import wandb

# Initialize your WandB API
api = wandb.Api()

# Define your project and sweep details
entity = "ravikumarchavva-org"  # Replace with your WandB organization or username
project = "T20I-CRICKET-WINNER-PREDICTION"
sweep_id = "qqakx1g3"  # The specific sweep ID

# Fetch the sweep
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")

# Retrieve all runs in the sweep
runs = sweep.runs

# Sort runs by a specific metric, e.g., validation accuracy (replace with your metric name)
# Use the metric key name you logged in your WandB runs
best_run = sorted(
    runs, key=lambda run: run.summary.get("val_accuracy", float("-inf")), reverse=True
)[1]

# Print details of the best run
print(f"Best run ID: {best_run.id}")
print(f"Validation Accuracy: {best_run.summary.get('val_accuracy')}")
config = best_run.config
config

Best run ID: yjilby5q
Validation Accuracy: 83.49514563106796


{'lr': 0.0001,
 'dropout': 0.6882847989323215,
 'batch_size': 32,
 'num_epochs': 100,
 'num_layers': 3,
 'hidden_size': 256,
 'enable_plots': False,
 'weight_decay': 5.754095511533712e-06,
 'learning_rate': 0.0009693209823947022}

In [3]:
# Get the model from run id
run_path = f"ravikumarchavva-org/T20I-CRICKET-WINNER-PREDICTION/{best_run.id}"

# Get the specific run
run = api.run(run_path)

# List and download output artifacts
for artifact in run.logged_artifacts():
    if artifact.name.startswith("best_model"):
        print(f"Downloading artifact: {artifact.name}")
        artifact_dir = artifact.download()
        print(f"Artifact downloaded to: {artifact_dir}")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb:   1 of 1 files downloaded.  


Artifact downloaded to: d:\github\Cricket-Prediction\ml_modeling\5_selecting_best_model\artifacts\best_model_val_loss_0.3524-v0


In [4]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(),"..",".."))

from utils.data_utils import collate_fn_with_padding, load_datasets, augument_data
from utils.model_utils import set_seed
from torch.utils.data import DataLoader

set_seed()
# Load the Datasets
train_dataset, val_dataset, test_dataset = load_datasets()

# Step 2: Augment Data
train_dataset, val_dataset, test_dataset = augument_data(train_dataset, val_dataset, test_dataset)

# Step 3: Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, collate_fn=collate_fn_with_padding)
val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, collate_fn=collate_fn_with_padding)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, collate_fn=collate_fn_with_padding)

wandb: Currently logged in as: ravikumarchavva (ravikumarchavva-org). Use `wandb login --relogin` to force relogin


In [5]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import os

# Load the model
model = torch.load(os.path.join(artifact_dir, 'best_model.pth'),weights_only=False)

# Evaluating on `test_dataset`

In [6]:
from utils.model_utils import evaluate_model

config['enable_plots'] = False

save_dir = os.path.dirname(os.getcwd())

# Define window sizes
window_sizes = [5,10,15,20, 25, 30, 35, 40, 45]

# Evaluate the model
metrics, all_labels, all_predictions, all_probs = evaluate_model(
    model, test_dataloader, device, window_sizes, config, save_dir=os.getcwd()
)
overall_metrics = metrics["overall_metrics"]
stage_metrics = metrics["stage_metrics"]
import pandas as pd
# Convert metrics to pandas DataFrames
stage_df = pd.DataFrame(stage_metrics).T
stage_df.index.name = "Stage"
stage_df.reset_index(inplace=True)

overall_df = pd.DataFrame(overall_metrics, index=["Overall"]).reset_index()
overall_df.rename(columns={"index": "Stage"}, inplace=True)

print("\nTest Data Metrics:")
# Print metrics in DataFrame format
print("Overall Metrics:")
print(overall_df.to_string(index=False))

print("\nStage Metrics:")
stage_df

Accuracy: 85.58 %

Test Data Metrics:
Overall Metrics:
  Stage  accuracy  precision   recall      f1
Overall  0.855769   0.924138 0.797619 0.85623

Stage Metrics:


,Stage,accuracy,precision,recall,f1
0,5 overs,0.700000,0.666667,0.500000,0.571429
1,10 overs,0.783333,0.869565,0.666667,0.754717
2,15 overs,0.811111,0.878049,0.750000,0.808989
3,20 overs,0.850000,0.900000,0.818182,0.857143
4,25 overs,0.860000,0.909091,0.833333,0.869565
5,30 overs,0.855556,0.913043,0.823529,0.865979
6,35 overs,0.861905,0.925234,0.825000,0.872247
7,40 overs,0.875000,0.928571,0.847826,0.886364
8,45 overs,0.877778,0.933333,0.840000,0.884211


In [24]:
# i = torch.randint(0, len(test_dataset), (1,)).item()
i = 0
train_dataset[i][0].unsqueeze(0).shape, train_dataset[i][1].unsqueeze(0).unsqueeze(0).shape, train_dataset[i][2].unsqueeze(0).shape

(torch.Size([1, 13]), torch.Size([1, 1, 22, 12]), torch.Size([1, 120, 10]))

In [8]:
print(train_dataset[i][0].shape[-1])

13


In [25]:
# Move the model to the device
model.to(device)

# Set the model to evaluation mode
model.eval()

# Visualize the model architecture
from torchinfo import summary  # Replace torchsummary with torchinfo
from torchviz import make_dot  # Add import for torchviz

# Visualize the model architecture using torchinfo
summary(model, input_size=[(1, 13), (1, 1, 22, 12), (1, 10)])

# Create a dummy input to visualize the graph
team_dummy = train_dataset[i][0].unsqueeze(0).to(device)
player_dummy = train_dataset[i][1].unsqueeze(0).to(device)
ball_dummy = train_dataset[i][2].unsqueeze(0).to(device)
# Forward pass to get the output
output = model(team_dummy, player_dummy, ball_dummy)

# Generate and save the model visualization
dot = make_dot(output, params=None)
dot.format = 'png'
dot.render('model_visualization')  # Saves as model_visualization.png

'model_visualization.png'

# Saving to ONNX format

In [26]:
val_dataset

In [27]:
# Export the model to ONNX

from utils.model_utils import export_model_to_onnx

export_path = os.path.join(os.getcwd(), "model.onnx")
export_model_to_onnx(model, export_path, (team_dummy, player_dummy, ball_dummy))

c:\Users\chavv\anaconda\envs\huggingface-torch\lib\site-packages\torch\onnx\symbolic_opset9.py:4545: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


Model has been exported to d:\github\Cricket-Prediction\ml_modeling\5_selecting_best_model\model.onnx


In [28]:
#import from onnx
import onnx
import onnxruntime as ort

# Load the ONNX model
onnx_model = onnx.load(export_path)

# Check the ONNX model
onnx.checker.check_model(onnx_model)

# Initialize the ONNX runtime session
ort_session = ort.InferenceSession(export_path)

# Get the input names
input_names = [input.name for input in ort_session.get_inputs()]

# Get the output names
output_names = [output.name for output in ort_session.get_outputs()]

# Print the input and output names
print(f"Input Names: {input_names}")
print(f"Output Names: {output_names}")

Input Names: ['team_input', 'player_input', 'ball_input']
Output Names: ['output']


In [68]:
i= torch.randint(0, len(val_dataset), (1,)).item()
team_input, player_input, ball_input,label = train_dataset[i]
team_input = team_input.unsqueeze(0).to(device)
player_input = player_input.unsqueeze(0).to(device)
ball_input = ball_input.unsqueeze(0).to(device)
label

tensor(1.)

In [69]:
onnx_input = {
    "team_input": team_input.cpu().numpy(),
    "player_input": player_input.cpu().numpy(),
    "ball_input": ball_input.cpu().numpy(),
}


In [70]:
# Run the ONNX model using the ONNX runtime session
outputs = ort_session.run(None, onnx_input)

# Print the outputs
print(outputs)

[array([[0.9979929]], dtype=float32)]
